In [0]:
 !pip3 install 'python-igraph'

In [0]:
import nltk
nltk.download('maxent_treebank_pos_tagger')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

#regarder ligne par ligne ordre de grander de cb d'operation on doit faire et on garde le pire (par ligne)

[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
import re 
import itertools
import operator
import copy
import igraph
import heapq
import nltk
# requires nltk 3.2.1
from nltk import pos_tag # nltk.download('maxent_treebank_pos_tagger')

def clean_text_simple(text, my_stopwords, punct, remove_stopwords=True, pos_filtering=True, stemming=True):
    text = text.lower()
    text = ''.join(l for l in text if l not in punct) # remove punctuation (preserving intra-word dashes)
    text = re.sub(' +',' ',text) # strip extra white space
    text = text.strip() # strip leading and trailing white space
    tokens = text.split(' ') # tokenize (split based on whitespace)
    if pos_filtering == True:
        # POS-tag and retain only nouns and adjectives
        tagged_tokens = pos_tag(tokens)
        tokens_keep = []
        for item in tagged_tokens:
            if (
            item[1] == 'NN' or
            item[1] == 'NNS' or
            item[1] == 'NNP' or
            item[1] == 'NNPS' or
            item[1] == 'JJ' or
            item[1] == 'JJS' or
            item[1] == 'JJR'
            ):
                tokens_keep.append(item[0])
        tokens = tokens_keep
    if remove_stopwords:
        tokens = [token for token in tokens if token not in my_stopwords]
    if stemming:
        stemmer = nltk.stem.PorterStemmer()
        tokens_stemmed = list()
        for token in tokens:
            tokens_stemmed.append(stemmer.stem(token))
        tokens = tokens_stemmed
    
    return(tokens)


def terms_to_graph(terms, window_size):
    '''This function returns a directed, weighted igraph from lists of list of terms (the tokens from the pre-processed text)
    e.g., ['quick','brown','fox']
    Edges are weighted based on term co-occurence within a sliding window of fixed size 'w'
    fenetre glissante met à jour liste de voisins, si deux mots apparaissent dans la meme fenetre, occurence +=1
    
    '''
    
    from_to = {}

    w = min(window_size, len(terms))
    # create initial complete graph (first w terms)
    terms_temp = terms[0:w]
    indexes = list(itertools.combinations(range(w), r=2))

    new_edges = []

    for my_tuple in indexes:
        new_edges.append(tuple([terms_temp[i] for i in my_tuple]))
    for new_edge in new_edges:
        if new_edge in from_to:
            from_to[new_edge] += 1
        else:
            from_to[new_edge] = 1

    # then iterate over the remaining terms
    for i in range(w, len(terms)):
        # term to consider
        considered_term = terms[i]
        # all terms within sliding window
        terms_temp = terms[(i - w + 1):(i + 1)]

        # edges to try
        candidate_edges = [] #possible nouvelles edges 
        for p in range(w - 1):
            candidate_edges.append((terms_temp[p], considered_term)) #terms temporaires = fenetre précédente

        for try_edge in candidate_edges:

            # if not self-edge
            if try_edge[1] != try_edge[0]:

                # if edge has already been seen, update its weight
                if try_edge in from_to:
                    from_to[try_edge] += 1

                # if edge has never been seen, create it and assign it a unit weight
                else:
                    from_to[try_edge] = 1

    # create empty graph
    g = igraph.Graph(directed=True)

    # add vertices
    g.add_vertices(sorted(set(terms)))

    # add edges, direction is preserved since the graph is directed
    g.add_edges(list(from_to.keys()))
    
    # set edge and vertice weights
    g.es['weight'] = list(from_to.values()) # based on co-occurence within sliding window
    g.vs['weight'] = g.strength(weights=list(from_to.values())) # weighted degree

    return (g)

''' As long as we have core left, we remove the element with the lowest value  '''
def core_dec(g,weighted):
    '''(un)weighted k-core decomposition'''
    # work on clone of g to preserve g 
    gg = copy.deepcopy(g)
    if not weighted:
        gg.vs['weight'] = gg.strength() # overwrite the 'weight' vertex attribute with the unweighted degrees
    # initialize dictionary that will contain the core numbers
    cores_g = dict(zip(gg.vs['name'],[0]*len(gg.vs)))
   # print(gg.vs['weight'])
    while len(gg.vs) > 0:
        # find index of lowest degree vertex
        min_degree = min(gg.vs['weight']) #minimum degre
        #print('MIN ', min_degree)
        index_top = gg.vs['weight'].index(min_degree) #index correspondant au poids min
        name_top = gg.vs[index_top]['name'] #mot correspondant au poids min
        # get names of its neighbors
        neighbors = gg.vs[gg.neighbors(index_top)]['name']
        # exclude self-edges
        neighbors = [elt for elt in neighbors if elt!=name_top]
        # set core number of lowest degree vertex as its degree
        cores_g[name_top] = min_degree
        ### delete top vertex and its incident edges ###
        gg.delete_vertices(gg.vs.select(name = name_top))
        if neighbors:
            if weighted: #On va parcourir les voisins du sommet supprimé et mettre à jour leur poids avec strenght()
                new_degrees = gg.strength(weights = gg.es['weight'])           
             
            else:
                #Parcourir les sommets voisins et compter leurs voisins respectifs
                new_degrees = gg.strength()
             # iterate over neighbors of top element
            for neigh in neighbors:
                index_n = gg.vs['name'].index(neigh)
                gg.vs[index_n]['weight'] = max(min_degree,new_degrees[index_n])  
                
    return(cores_g)


def accuracy_metrics(candidate, truth):
    # true positives ('hits') are both in candidate and in truth
    tp = len(set(candidate).intersection(truth))
    # false positives a.k.a. false alarms are in candidate but not in truth
    fp = len([element for element in candidate if element not in truth])
    ### compute false negatives a.k.a. misses, save results as 'fn'
    fn = len([element for element in truth if element not in candidate])
    ### compute precision and recall as a function of 'tp', 'fp' and 'fn', save results as 'prec' and 'rec' ###
    prec = tp/(tp+fp)
    rec = tp/(tp+fn)
    if prec+rec != 0:
        f1 = 2*prec*rec/(prec+rec)
    else:
        f1 = 0
    
    return (prec, rec, f1)


In [0]:

## GOW TOY ##
import string
!nltk.download('stopwords')
from nltk.corpus import stopwords


#import os
#os.chdir() # to change working directory to where functions live
# import custom functions
#from library import clean_text_simple, terms_to_graph, core_dec

stpwds = stopwords.words('english')
punct = string.punctuation.replace('-', '')

my_doc = 'A method for solution of systems of linear algebraic equations \
with m-dimensional lambda matrices. A system of linear algebraic \
equations with m-dimensional lambda matrices is considered. \
The proposed method of searching for the solution of this system \
lies in reducing it to a numerical system of a special kind.'

my_doc = my_doc.replace('\n', '')

# pre-process document
my_tokens = clean_text_simple(my_doc,my_stopwords=stpwds,punct=punct)

g = terms_to_graph(my_tokens, 4) #window of size 4

# number of edges
print(len(g.es))

# the number of nodes should be equal to the number of unique terms
len(g.vs) == len(set(my_tokens))

edge_weights = []
for edge in g.es:
    source = g.vs[edge.source]['name']
    target = g.vs[edge.target]['name']
    weight = edge['weight']
    edge_weights.append([source, target, weight])

print(edge_weights)

for w in range(2,10):
    g = terms_to_graph(my_tokens, w)
    ### print density of g ###
    print('density ',g.density(loops=False))

# decompose g
core_numbers = core_dec(g,True)
print('core numbers', core_numbers)
print('coreness',g.coreness(mode=3))
### compare 'core_numbers' with the output of the .coreness() igraph method ###

# retain main core as keywords
max_c_n = max(core_numbers.values())
keywords = [kwd for kwd, c_n in core_numbers.items() if c_n == max_c_n]
print("keys",keywords)


/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'
42
[['method', 'solut', 2], ['method', 'system', 2], ['method', 'linear', 1], ['solut', 'system', 3], ['solut', 'linear', 1], ['system', 'linear', 2], ['solut', 'algebra', 1], ['system', 'algebra', 2], ['linear', 'algebra', 2], ['system', 'equat', 2], ['linear', 'equat', 2], ['algebra', 'equat', 2], ['linear', 'm-dimension', 2], ['algebra', 'm-dimension', 2], ['equat', 'm-dimension', 2], ['algebra', 'lambda', 2], ['equat', 'lambda', 2], ['m-dimension', 'lambda', 2], ['equat', 'system', 1], ['m-dimension', 'system', 1], ['lambda', 'system', 1], ['m-dimension', 'linear', 1], ['lambda', 'linear', 1], ['lambda', 'algebra', 1], ['equat', 'matric', 1], ['m-dimension', 'matric', 1], ['lambda', 'matric', 1], ['m-dimension', 'method', 1], ['lambda', 'method', 1], ['matric', 'method', 1], ['lambda', 'solut', 1], ['matric', 'solut', 1], ['matric', 'system', 1], ['method', 'n

In [0]:
###KEY EXTRACTION###

import os
import string
import re 
import operator
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

#from library import clean_text_simple,terms_to_graph,core_dec,accuracy_metrics

stemmer = nltk.stem.PorterStemmer()
stpwds = stopwords.words('english')
punct = string.punctuation.replace('-', '')

##################################
# read and pre-process abstracts #
##################################

path_to_abstracts =r"for_moodle/data/Hulth2003testing/abstracts"
abstract_names = sorted(os.listdir(path_to_abstracts))

abstracts = []
for counter,filename in enumerate(abstract_names):
    # read file
    with open(path_to_abstracts + '/' + filename, 'r') as my_file: 
        text = my_file.read().splitlines()
    text = ' '.join(text)
    # remove formatting
    text = re.sub('\s+', ' ', text)
    abstracts.append(text)
    
    if counter % round(len(abstract_names)/5) == 0:
        print(counter, 'files processed')

abstracts_cleaned = []
for counter,abstract in enumerate(abstracts):
    my_tokens = clean_text_simple(abstract,my_stopwords=stpwds,punct=punct)
    abstracts_cleaned.append(my_tokens)
    
    if counter % round(len(abstracts)/5) == 0:
        print(counter, 'abstracts processed')

###############################################
# read and pre-process gold standard keywords #
###############################################

path_to_keywords = r"for_moodle/data/Hulth2003testing/uncontr/"
keywd_names = sorted(os.listdir(path_to_keywords))
   
keywds_gold_standard = []

for counter,filename in enumerate(keywd_names):
    # read file
    with open(path_to_keywords + filename, 'r') as my_file: 
        text = my_file.read().splitlines()
    text = ' '.join(text)
    text =  re.sub('\s+', ' ', text) # remove formatting
    text = text.lower()
    # turn string into list of keywords, preserving intra-word dashes 
    # but breaking n-grams into unigrams
    keywds = text.split(';')
    keywds = [keywd.strip().split(' ') for keywd in keywds]
    keywds = [keywd for sublist in keywds for keywd in sublist] # flatten list
    keywds = [keywd for keywd in keywds if keywd not in stpwds] # remove stopwords (rare but may happen due to n-gram breaking)
    keywds_stemmed = [stemmer.stem(keywd) for keywd in keywds]
    keywds_stemmed_unique = list(set(keywds_stemmed)) # remove duplicates (may happen due to n-gram breaking)
    keywds_gold_standard.append(keywds_stemmed_unique)
    
    if counter % round(len(keywd_names)/5) == 0:
        print(counter, 'files processed')

##############################
# precompute graphs-of-words #
##############################

### use the terms_to_graph function, store the results in a list named 'gs' ##
gs = []
for counter in enumerate(abstracts_cleaned):
    terms = terms_to_graph(counter[1], 4) #1 car on veut que la partie texte
    gs.append(terms)
    
#gs = [terms_to_graph(abstracts_cleaned[i], 4) for i in range(len(abstracts_cleaned))]
    
##################################
# graph-based keyword extraction #
##################################

my_percentage = 0.33 # for PR and TF-IDF

method_names = ['kc','wkc','pr','tfidf']
keywords = dict(zip(method_names,[[],[],[],[]]))

for counter,g in enumerate(gs):
    # k-core
    core_numbers = core_dec(g,False)
    ### retain main core as keywords and append the resulting list to 'keywords['kc']' ###
    main_core = max(core_numbers.values()) #On récupère la valeur du plus gros poids
    keyword = [kwd for kwd, c_n in core_numbers.items() if c_n == main_core] #les keywords seront tous les sommets de poids max
    keywords['kc'].append(keyword)
 
    # weighted k-core
    core_numbers = core_dec(g,True)
    ### retain main core as keywords and append the resulting list to 'keywords['kc']' ###
    main_core = max(core_numbers.values())
    keyword = [kwd for kwd, c_n in core_numbers.items() if c_n == main_core]
    keywords['wkc'].append(keyword)
    

    # PageRank
    pr_scores = zip(g.vs['name'],g.pagerank())
    pr_scores = sorted(pr_scores, key=operator.itemgetter(1), reverse=True) # in decreasing order
    numb_to_retain = int(len(pr_scores)*my_percentage) # retain top 'my_percentage' % words as keywords
    keywords['pr'].append([tuple[0] for tuple in pr_scores[:numb_to_retain]]) 
    if counter % round(len(gs)/5) == 0:
        print(counter)

#############################
# TF-IDF keyword extraction #
#############################

abstracts_cleaned_strings = [' '.join(elt) for elt in abstracts_cleaned] # to ensure same pre-processing as the other methods
tfidf_vectorizer = TfidfVectorizer(stop_words=stpwds)
### call the .fit_transform() method and name the result 'doc_term_matrix' ###
doc_term_matrix = tfidf_vectorizer.fit_transform(abstracts_cleaned_strings)
terms = tfidf_vectorizer.get_feature_names() #Les mots
vectors_list = doc_term_matrix.todense().tolist()

for counter,vector in enumerate(vectors_list):
    terms_weights = zip(terms,vector) # bow feature vector as list of tuples
    ### keep only non zero values, i.e., the words in the document. store the results in a list named 'nonzero' ##
    nonzero = list(terms_weights)
    nonzero = [elt for elt in nonzero if elt[1] != 0.0 ]  
    nonzero = sorted(nonzero, key=operator.itemgetter(1), reverse=True) # in decreasing order
    numb_to_retain = int(len(nonzero)*my_percentage) # retain top 'my_percentage' % words as keywords
    keywords['tfidf'].append([tuple[0] for tuple in nonzero[:numb_to_retain]])
    
    
    if counter % round(len(vectors_list)/5) == 0:
        print(counter)

##########################
# performance comparison #
##########################

perf = dict(zip(method_names,[[],[],[],[]]))
for idx,truth in enumerate(keywds_gold_standard):
    for mn in method_names:
        ### append to the 'perf[mn]' list by using the 'accuracy_metrics' function ###
        accuracy = accuracy_metrics(keywords[mn][idx], truth)
        perf[mn].append(accuracy)
lkgs = len(keywds_gold_standard)

# print macro-averaged results (averaged at the collection level)
for k,v in perf.items():
    print(k + ' performance: \n')
    print('precision:', round(100*sum([tuple[0] for tuple in v])/lkgs,2))
    print('recall:', round(100*sum([tuple[1] for tuple in v])/lkgs,2))
    print('F-1 score:', round(100*sum([tuple[2] for tuple in v])/lkgs,2))
    print('\n')

0 files processed
100 files processed
200 files processed
300 files processed
400 files processed
0 abstracts processed
100 abstracts processed
200 abstracts processed
300 abstracts processed
400 abstracts processed
0 files processed
100 files processed
200 files processed
300 files processed
400 files processed
0
100
200
300
400
0
100
200
300
400
kc performance: 

precision: 51.86
recall: 62.56
F-1 score: 51.55


wkc performance: 

precision: 63.86
recall: 48.64
F-1 score: 46.52


pr performance: 

precision: 60.18
recall: 38.3
F-1 score: 44.96


tfidf performance: 

precision: 59.21
recall: 38.5
F-1 score: 44.85


